In [1]:
import os
%pwd
os.chdir("../")

In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class model_eval_config:
    root_dir: Path




In [ ]:
from src.Mini_Translator.constants import *
from src.Mini_Translator.utils.common import read_yaml, create_directories


In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_eval_model_config(self) -> model_eval_config:
        config = self.config.model_evaluation
        params=self.params

        create_directories([config.root_dir])

        eval_config=model_eval_config(root_dir=config.root_dir)
        
        return eval_config


In [ ]:
import torch
import numpy as np
import json
import torch.nn as nn
from src.Mini_Translator.logging import logger
import spacy
import pandas as pd
import datasets
from datasets import Dataset, DatasetDict
import evaluate


import tqdm

In [ ]:
class Evaluation:
    def __init__(self,config:model_eval_config,config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):

        
        self.params = read_yaml(params_filepath)
        self.config=config
        self.config2=config_filepath

 
    
    def evaluate_fn(self,model,data_loader,criterion,device):
        model.eval()
        epoch_loss=0
        with torch.no_grad():
            for i,batch in enumerate(data_loader):
                src=batch["de_ids"].to(device)
                trg=batch["en_ids"].to(device)
                output=model(src,trg,0)
                output_dim=output.shape[-1]
                output=output[1:].view(-1,output_dim)
                trg=trg[1:].view(-1)
                loss=criterion(output,trg)
                epoch_loss+=loss.item()
        return epoch_loss / len(data_loader)
    
    def translate_sentence(self,
    sentence,
    model,
    en_nlp,
    de_nlp,
    en_vocab,
    de_vocab,
    lower,
    sos_token,
    eos_token,
    device,
    max_output_length=25,
    ):
        model.eval()
        with torch.no_grad():
            if isinstance(sentence, str):
                tokens = [token.text for token in de_nlp.tokenizer(sentence)]
            else:
                tokens = [token for token in sentence]
            if lower:
                tokens = [token.lower() for token in tokens]
            tokens = [sos_token] + tokens + [eos_token]
            ids = de_vocab.lookup_indices(tokens)
            tensor = torch.LongTensor(ids).unsqueeze(-1).to(device)
            hidden, cell = model.encoder(tensor)
            inputs = en_vocab.lookup_indices([sos_token])
            for _ in range(max_output_length):
                inputs_tensor = torch.LongTensor([inputs[-1]]).to(device)
                output, hidden, cell = model.decoder(inputs_tensor, hidden, cell)
                predicted_token = output.argmax(-1).item()
                inputs.append(predicted_token)
                if predicted_token == en_vocab[eos_token]:
                    break
            tokens = en_vocab.lookup_tokens(inputs)
        return tokens
    

    def initiate_model_Evaluation(self):
        root_dir = "artifacts/data_transformation"

        test_data_loader_path = os.path.join(root_dir, "test_data_loader.pth")
        test_data_loader = torch.load(test_data_loader_path)
        

        with open("metadata.json",'r') as file:
            f=json.load(file)
            pad_index=f.pad_index
            device=f.device
        criterion=nn.CrossEntropyLoss(ignore_index=pad_index)

        model.load_state_dict(torch.load("tut1-model.pt"))

        test_loss = self.evaluate_fn(model, test_data_loader, criterion, device)

        print(f"| Test Loss: {test_loss:.3f} | Test PPL: {np.exp(test_loss):7.3f} |")


        en_nlp = spacy.load(self.config2.data_transformation.tokenizer_1)
        de_nlp = spacy.load(self.config2.data_transformation.tokenizer_2)


        en_vocab = torch.load('en_vocab.pth')
        de_vocab = torch.load('de_vocab.pth')

        with open(self.config2.data_ingestion.data_files.test, 'r') as json_file:
            test_data = json.load(json_file)

        test_data = Dataset.from_pandas(pd.DataFrame(test_data))
        
        translations = [
            self.translate_sentence(
                example["de"],
                model,
                en_nlp,
                de_nlp,
                en_vocab,
                de_vocab,
                self.params.lower,
                self.params.sos_token,
                self.params.eos_token,
                device,
            )
            for example in tqdm.tqdm(test_data)
        ]

        bleu = evaluate.load("bleu")

        predictions = [" ".join(translation[1:-1]) for translation in translations]

        references = [[example["en"]] for example in test_data]

        def get_tokenizer_fn(nlp, lower):
            def tokenizer_fn(s):
                tokens = [token.text for token in nlp.tokenizer(s)]
                if lower:
                    tokens = [token.lower() for token in tokens]
                return tokens

            return tokenizer_fn
        tokenizer_fn = get_tokenizer_fn(en_nlp, self.params.lower)

        results = bleu.compute(
            predictions=predictions, references=references, tokenizer=tokenizer_fn
        )

        json_path = os.path.join(self.config.root_dir,"results.json")

        # Save the results to a JSON file
        with open(json_path, 'w') as f:
            json.dump(results, f, indent=4)

        logger.info(f"Results saved to {json_path}")

In [ ]:
try:
    config = ConfigurationManager()
    eval_model_config = config.get_eval_model_config()
    model = Evaluation(config=eval_model_config)
    model.initiate_model_Evaluation()
except Exception as e:
    raise e